# Augmentation Tool

The augmentation tool can be used to augment an image set organized as corpus within a set of corpora.

An image corpus itself can consist of several folders. Each folder represents a class the respective classifier will be able to recognize. Each class folder contains all images that will be used to train and test the classifier on this class. If only one class per classifier is given, a folder called negative_examples is needed as well.

To get a better understanding of the layout, take a look at this sample folder hierarchy (also contained in this project):

```
 ./corpus
     /audi
         /athree
             3_1.jpg
             ...
         /afour
             a4_1.jpg
             ...
     /mercedes
         /sclass
             sclass_1.jpg
             ...
         /negative_examples
             negative_sclass_1.jpg
             ...
```

# Initialization

In [ ]:
# import basic libraries
import numpy as np
import pandas as pd
import os
import sys

import vrtool

# Corpora Overview

In [ ]:
# The name of the folder that contains the corpora, currently relative to notebook location
corpora_folder_name = '../corpus'
config_name = 'config.ini'

runner = vrtool.Runner(corpora_folder_name, config_name)
corpora = runner.get_available_corpora()

# Print a summary of the available corpora in the corpora directory
print('\nAvailable image corpora:')
print('\n'.join('{}: {}'.format(*el) for el in enumerate(corpora)))
print()

for corpus_name, corpus_dir in corpora:
    if(not "negative" in corpus_name):
        imgs = corpus_dir.get_all_class_images()
        image_info = pd.DataFrame(imgs)
        try:
            print('---------------------------------')
            print("Corpus:",corpus_name)
            print(image_info.class_name.value_counts())
        except:
            pass

# Augmentation Corpus Configuration
The **AUGMENTATION_CORPUS_NAME** defines the corpus that will be augmented.
The augmentation pipeline performs the following tasks: 
- mirror image, 
- rotate left, 
- rotate right

If a class contains less than **MIN_IMAGE_COUNT** images, it will be augmented

The **DEGREE** constant defined the absolute angle that will be used for rotation.

In [ ]:
# The name of the corpus to be augmented
AUGMENTATION_CORPUS_NAME = "mercedes"
# The rotation angle (postive and negative) 
DEGREE = 10
# If a subclass contains less than min_image_count it will be augmented 
MIN_IMAGE_COUNT = 50

# Iterate through all images in corpus folder and perform image augmentation
corpora_info = []
for corpus_name, corpus_dir in corpora:
    if(not "negative" in corpus_name):
        imgs = corpus_dir.get_all_class_images()
        image_info = pd.DataFrame(imgs)
        corpora_info.append({'corpus_name': corpus_name, 'corpus_dir': corpus_dir, 'image_info':image_info })
        try:
            if (corpus_name == AUGMENTATION_CORPUS_NAME):
                # Rotate images - x°, + x° and mirror them if the corpus contains less than y images
                runner.duplicate_all(corpus_dir, DEGREE, MIN_IMAGE_COUNT)
                print('---- Before Augmentation -----------------------------')
                print("Corpus:",corpus_name)
                print(image_info.class_name.value_counts())
                
                imgs = corpus_dir.get_all_class_images()
                image_info = pd.DataFrame(imgs)
                print('---- After Augmentation ------------------------------')
                print("Corpus:",corpus_name)
                print(image_info.class_name.value_counts())
            
        except AttributeError:
            pass   